In [52]:
import numpy as np
from enum import IntEnum


class Cards(IntEnum):
    DUKE = 0
    ASSA = 1
    CAPT = 2
    AMBA = 3
    CONT = 4
    
    def __str__(self):
        return self.name
    
class Actions(IntEnum):
    INCOME = 0
    FOREIGN_AID = 1
    COUP = 2
    TAX = 3
    ASSASSINATE = 4
    STEAL = 5
    EXCHANGE = 6
    
    def __str__(self):
        return self.name

class Counteractions(IntEnum):
    BLOCK_STEAL = 0
    BLOCK_ASSASSINATE = 1
    BLOCK_FOREIGN_AID = 2       

class Rules:
    STARTING_COINS = 2
    INCOME_AMT = 1
    FOREIGN_AID_AMT = 2
    TAX_AMT = 3
    STEAL_AMT = 2
    COUP_COST = 7
    ASSASSINATE_COST = 3
    
    
class Action_Response(IntEnum):
    PASS = 0
    CHALLENGE = 1
    COUNTERACT = 2
    
    def __str__(self):
        return self.name



DECK_LAYOUT = {
    Cards.DUKE : 3,
    Cards.ASSA : 3,
    Cards.CAPT : 3,
    Cards.AMBA : 3,
    Cards.CONT : 3
}

ALL_ACTIONS = list(Actions)

INCONTESTABLE_ACTIONS = [Actions.INCOME, Actions.COUP]




class Player:
    def __init__(self, id):
        self.id = id


def log(func):
    def wrapper(*args, **kwargs):
        print(f"[LOG] Calling {func.__name__} with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"[LOG] {func.__name__} returned {result}")
        return result
    return wrapper
    


class CoupEnv:
    def __init__(self, num_players: int = 3, seed: int | None = None):
        """Initialize environment with given number of players."""
        self.rng = np.random.default_rng(seed)
        self.num_players = num_players
        #self.players = {i : Player(i) for i in range(num_players)}
        self.deck = []
        self.reset()

    def reset(self):

        self.deck.clear()

        #supply the deck with cards according to their frequency
        for card,freq in DECK_LAYOUT.items():
            for _ in range(freq):
                self.deck.append(card)
        
        self.player_states = {}

        #dole each player two cards and two coins to start the game
        for i in range(self.num_players):

            card_1 = self._deal_card()
            card_2 = self._deal_card()

            
            self.player_states[i] = {
                "coins" : Rules.STARTING_COINS,
                "cards" : [card_1, card_2],
                "alive" : True
            }
   
    @log
    def apply_action(self, action, no_effect=False):
        #action should be {"player_id": id, "action_type" : int, "target_player_id": None}
        
        action_function = self._get_action_function(action)
        
        action_function(no_effect)

    
    def _get_action_function(self, action):

        action_type = action["action_type"]
        player_id = action["player_id"]
        target_player_id = action["target_player_id"]
        
        if action_type == Actions.INCOME:
            return lambda no_effect: self._mod_player_coins(player_id, change = Rules.INCOME_AMT)
            
        elif action_type == Actions.FOREIGN_AID:
            return lambda no_effect: self._mod_player_coins(player_id, change = Rules.FOREIGN_AID_AMT)

        elif action_type == Actions.COUP:
            def coup_function(no_effect):
                self._mod_player_coins(player_id, change = -Rules.COUP_COST)
                self.request_card_reveal(target_player_id)
            return coup_function
            
        elif action_type == Actions.TAX:
            return lambda no_effect: self._mod_player_coins(player_id, change = Rules.TAX_AMT)
            
        elif action_type == Actions.ASSASSINATE:
            def assassinate_function(no_effect):
                self._mod_player_coins(player_id, change = -Rules.ASSASSINATE_COST)
                self.request_card_reveal(target_player_id)
            return assassinate_function

        elif action_type == Actions.STEAL:
            def steal_function(no_effect):
                coins_to_steal = min(Rules.STEAL_AMT, self.player_states[target_player_id]["coins"])
                self._mod_player_coins(target_player_id, change = -coins_to_steal)
                self._mod_player_coins(player_id, change = coins_to_steal)
            return steal_function
            
        elif action_type == Actions.EXCHANGE:
            def exchange_function(no_effect):
                self.request_exchange(player_id)
            return exchange_function            
        
        else:
            raise Exception("invalid input")    
            
            
        
            
    def _mod_player_coins(self, player_id, change):
        if (self.player_states[player_id]["coins"] + change) < 0:
            raise Exception("action invalid- coins goes below 0")
        
        self.player_states[player_id]["coins"] += change

    def give_player_new_card(self, player_id):
        
        assert(len(self.player_states[player_id]["cards"]) == 1), "Player has too many cards or is out." 
        
        new_card = self._deal_card()

        self.player_states[player_id]["cards"].append(new_card)
        
        
    

    #functions that require user input\

    @log
    def request_player_action(self, player_id):
        actions = self._get_legal_actions(player_id)
        action_type = Actions(self.rng.choice(actions))

        target_id = None
        if action_type in [Actions.COUP, Actions.STEAL, Actions.ASSASSINATE]:
            
            target_id = int(self.rng.choice(self.get_targets(player_id)))

        return {"player_id": player_id, "action_type": action_type, "target_player_id":target_id}

    @log
    def request_card_reveal(self, player_id):

        #todo get user input instead of taking the last element
        state = self.player_states[player_id]
        
        
        revealed_card = self.player_states[player_id]["cards"].pop()
        

        return revealed_card

    @log
    def request_exchange(self, player_id):

        player_cards = self.player_states[player_id]["cards"]
        num_player_cards = len(player_cards)
        
        #todo get user input
        first = self._deal_card()
        second = self._deal_card()
        
        show_these = [first, second] + player_cards

        #todo show these cards to the player and let them choose
        user_picked = self.rng.choice(show_these, num_player_cards)
        
        to_go_back = set(show_these) - set(user_picked)
        
        for card in to_go_back:
            self.deck.append(card)

        self.player_states[player_id]["cards"] = list(user_picked) #because rng returns np array

        
    def propose_action(self, action) -> dict | None:
        # prompts all players for their response. The first non pass shorts the function and is returned
        # contest is a None, or a dict of {"type": in [challenge, counteract] , 
        # "from_player:int": player_id, "counteraction" : in [block foreign aid, block steal, block assassinate]}
        
        
        return None
    #end functions that require user input

    
    
    def _get_legal_actions(self, player_id: int) -> list[int]:
        """Return list of legal actions for the given player, considering coins and board state."""
        state = self.player_states[player_id]

        if state["coins"] >= 10:
            return [Actions.COUP]

        actions = ALL_ACTIONS.copy()
        
        
        if state["coins"] < 7:
            actions.remove(Actions.COUP)
            
            if state["coins"] < 3:
                actions.remove(Actions.ASSASSINATE)
        
        return actions
    @log
    def get_targets(self, player_id:int) -> list[int]:
        return [id for id in self.player_states.keys() if id != player_id and self.player_states[id]["alive"]]

    def _deal_card(self)-> int:
        assert(self.deck and len(self.deck) > 0), "Deck Empty or Not Yet Initialized"

        card = Cards(self.rng.choice(self.deck))
        self.deck.remove(card)

        return card
        
    def update_player_lives(self):
        for id,player in self.player_states.items():
            if (len(player["cards"]) == 0) and (player["alive"]):
                self.player_states[id]["alive"] = False
                print(f"player {id} died")
        

    def is_terminal(self) -> bool:

        #check that all but one player is dead
        one_player_alive = False
        
        for state in self.player_states.values():
            
            if one_player_alive and state['alive']:
                return False
            
            if state["alive"]:
                one_player_alive = True
        
        return True
            
            
        """Check if the game is over."""

    

In [49]:
def take_turn(player_id,env):
    
    
    print(f"player {player_id} is {"alive " if  env.player_states[player_id]["alive"] else "dead"}")
    
    if not env.player_states[player_id]["alive"]: return
    
    print(f"has {len(env.player_states[player_id]["cards"])} {env.player_states[player_id]["cards"]} and {env.player_states[player_id]["coins"]} coins")
    
    curr_player_action = env.request_player_action(player_id)


    curr_player_action_type = curr_player_action["action_type"]
    
    print(f"player {player_id} tries action: {curr_player_action_type}")

    if curr_player_action_type in INCONTESTABLE_ACTIONS:
        env.apply_action(curr_player_action)


        print("action uncontestable therefore passed")
        
        # next turn
        return
    

    # else action is contestable

    contest = env.propose_action(curr_player_action) #prompts all players for their response. The first non pass shorts the function and is returned
    #contest is a None, or a dict of {"type": in [challenge, counteract] , "from_player:int": player_id, "counteraction" : in [block foreign aid, block steal, block assassinate]}


    if not contest:
        env.apply_action(curr_player_action)


        print("action uncontested therefore passed")
        # next turn
        return

    contest_type = contest["type"]
    challenger = contest["from_player"]
    
    if contest_type == CHALLENGE:
        
        revealed_card = env.request_card_reveal(player_id) # p is prompted to reveal a card
        
        if revealed_card == necessary_card_for_action[curr_player_action_type]: #where necessary card is a dict mapping an action to the card needed for it -- will
            #p wins the challenge
            
            penalty_card = env.request_card_reveal(challenger) # challenger has to reveal a card if their challenge is wrong
            env.give_player_new_card(player_id) #p gets a new card
            env.deck.append(revealed_card) # the old card has to go back into the deck
            env.apply_action(curr_player_action) #action goes through anyway

            print("action challenged but challenge failed therefore passed")
        #else ie if revealed card shows p's bluff or they choose not to reveal their card
        # accuser wins challenge
        # nothing has to happen since request_card_reveal removed the card from the player p. We don't apply the action, just pass to the next turn
        print("action discarded")

        # next turn
        return
        
    #an action and counteraction have a 1:1 relationship, right? 
    if contest_type == COUNTERACT:

        counteraction = contest["counteraction"]
        counteract_challenge = env.propose_action(counteraction) #prompt all other players for a response to the counteraction
        
        if not counteract_challenge:
            env.apply_action(curr_player_action, no_effect = True) #the action is blocked so it should cost p money but have no effect

            print("action blocked because counteract was not challenged")
            # next turn
            return
            
        
        #else it is challenged ie. assassin from p, blocked by contessa from counteracter, someone says you don't have contessa 
        revealed_counteraction_card = env.request_card_reveal(challenger) # ask the counteractor to reveal a card
        
        if revealed_counteraction_card in necessary_card_for_counteraction[counteraction]:
            
            penalty_card = env.request_card_reveal(counteract_challenge["from_player"])
            env.give_player_new_card(challenger) #the successful counteractor gets a new card
            env.deck.append(revealed_counteraction_card)
            env.apply_action(curr_player_action, no_effect = True) 
            print("action blocked because counteract was challenge and passed")
            
            # next turn
            return

        #else the counteractor bluffed, they dont get a card back after revealing it
        
        env.apply_action(curr_player_action)
        

In [51]:
#main loop

num_players= 3

necessary_card_for_action = {Actions.TAX : Cards.DUKE, Actions.ASSASSINATE: Cards.ASSA, Actions.STEAL: Cards.CAPT, Actions.EXCHANGE: Cards.AMBA}
necessary_card_for_counteraction = {BLOCK_FOREIGN_AID : [Cards.DUKE], BLOCK_ASSASSINATE: [Cards.CONT], BLOCK_STEAL: [Cards.CAPT,Cards.AMBA]}




turn_order = [i for i in range(num_players)]

#Setup game
env = CoupEnv(num_players= num_players)
game_over = False


#win state should be checked after every turn ? 

turn_num = 0
while not game_over:
    for curr_player_id in turn_order:
        print(f"Turn number {turn_num}: player {curr_player_id} go:" )
        take_turn(curr_player_id,env)
        env.update_player_lives()
        

        if env.is_terminal():
            game_over = True
            break
        
    turn_num+=1


        
			

		

    




Turn number 0: player 0 go:
player 0 is alive 
has 2 [<Cards.AMBA: 3>, <Cards.CONT: 4>] and 2 coins
[LOG] Calling request_player_action with args=(<__main__.CoupEnv object at 0x10ab35f90>, 0), kwargs={}
[LOG] request_player_action returned {'player_id': 0, 'action_type': <Actions.INCOME: 0>, 'target_player_id': None}
player 0 tries action: INCOME
[LOG] Calling apply_action with args=(<__main__.CoupEnv object at 0x10ab35f90>, {'player_id': 0, 'action_type': <Actions.INCOME: 0>, 'target_player_id': None}), kwargs={}
[LOG] apply_action returned None
action uncontestable therefore passed
Turn number 0: player 1 go:
player 1 is alive 
has 2 [<Cards.CONT: 4>, <Cards.CAPT: 2>] and 2 coins
[LOG] Calling request_player_action with args=(<__main__.CoupEnv object at 0x10ab35f90>, 1), kwargs={}
[LOG] request_player_action returned {'player_id': 1, 'action_type': <Actions.TAX: 3>, 'target_player_id': None}
player 1 tries action: TAX
[LOG] Calling apply_action with args=(<__main__.CoupEnv object at 

In [12]:


class GameClient:
    
    MAX_ATTEMPTS = 10
    
    def __init__(self):
        pass
    
    def _tell_user(self,message):
        print(message)
    
    def _get_user_input(self):
        return input()
    
    
    def _normalize_list(self,lst:list) -> list:
        normalized_list = []
        for word in lst:
            normalized_list.append(self._normalize_text(word))
        return normalized_list
            
    def _normalize_text(self,word:str) -> str:
        return word.lower()
        
    
    def ask_input(self,options:list):
        
        
        self._tell_user("Choose from these options:" + ", ".join(options))
        choice = self._get_user_input()
        
        choice = self._normalize_text(choice)
        
        attempts = 1
        while (attempts < GameClient.MAX_ATTEMPTS) and (choice not in self._normalize_list(options)):
            self._tell_user("Invalid choice. Try again.")
            choice = self._get_user_input()
            attempts+=1
        
        return choice
        
        
        
    

In [ ]:
class GameController:
    
    def __init__(self, env):
        

'hi'